In [101]:
import os
import json
import cv2
import numpy as np
from glob import glob
import subprocess

In [117]:
def rotate_instrinsics_90cw(K, image_shape):
    h,w = image_shape[:2]
    new_K = np.zeros_like(K)

    new_K[0, 0] = K[1, 1]
    new_K[1,1] = K[0, 0]
    new_K[0, 2] = h - K[1, 2]
    new_K[1, 2] = K[0, 2]
    new_K[2, 2] = 1.0

    return new_K

def pose_rotation(R):
    R_z_90cw = np.array([
    [0, 1, 0],
    [-1, 0, 0],
    [0, 0, 1]
    ])

    return R_z_90cw @ R

def generate_rotated_folders(palms_root="../PALMS+", output_root="../Panorama"):
    session_paths = sorted(glob(os.path.join(palms_root, "*", "Session_*")))

    for session_path in session_paths:
        building = os.path.basename(os.path.dirname(session_path))
        session_name = os.path.basename(session_path)

        # Create output directory: Panorama/Building/Session_*
        out_dir = os.path.join(output_root, building, session_name)
        os.makedirs(out_dir, exist_ok=True)

        image_dir = os.path.join(session_path, "images")
        image_paths = sorted(glob(os.path.join(image_dir, "image_*.png")))

        print(f"🔁 Processing {building}/{session_name} ({len(image_paths)} images)")

        for img_path in image_paths:
            img = cv2.imread(img_path)
            if img is None:
                print(f"⚠️ Could not read image: {img_path}")
                continue

            rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            img_name = os.path.basename(img_path)
            out_path = os.path.join(out_dir, img_name)

            cv2.imwrite(out_path, rotated)

        print(f"Saved rotated images to {out_dir}")


In [77]:
def get_pano_file_paths():
    base_path="../Palms+/"
    pattern = os.path.join(base_path, "*/Session_*")
    pano_folders = sorted(glob(pattern))
    return pano_folders

def get_image_paths(pano_path):
    """
    Given a PALMS+ session folder, return sorted list of full image paths.
    Assumes images are in a subfolder called 'images' and named image_*.png
    """
    image_dir = os.path.join(pano_path, "images")
    image_paths = sorted(glob(os.path.join(image_dir, "image_*.png")))
    return image_paths

def get_rotated_image_paths_from_palms_path(palms_session_path):
    """
    Given a path like '../PALMS+/BE/Session_1744229291',
    return rotated image paths from './BE/Session_1744229291' assuming you're in Panorama/.
    """
    building = os.path.basename(os.path.dirname(palms_session_path))
    session = os.path.basename(palms_session_path)
    panorama_session_path = os.path.join(building, session)
    image_paths = sorted(glob(os.path.join(panorama_session_path, "image_*.png")))
    return image_paths

def get_image_folder(pano_path):
    image_dir = os.path.join(pano_path, "images")
    image_paths = sorted(glob(os.path.join(image_dir, "image_*.png")))
    images = [cv2.rotate(cv2.imread(p), cv2.ROTATE_90_CLOCKWISE) for p in image_paths]
    return images, image_paths

def get_intrinsics(pano_path):
    intrinsics_dir = os.path.join(pano_path, "intrinsics")
    intrinsic_paths = sorted(glob(os.path.join(intrinsics_dir, "cameraIntrinsics_*.json")))
    intrinsics = []
    for path in intrinsic_paths:
        with open(path) as f:
            data = json.load(f)["data"]
            K = np.array(data).reshape(3, 3)
            intrinsics.append(K)
    return intrinsics

def get_rotations(pano_path):
    poses_dir = os.path.join(pano_path, "poses")
    pose_paths = sorted(glob(os.path.join(poses_dir, "cameraPose_*.json")))
    rotations = []
    for path in pose_paths:
        with open(path) as f:
            pose_data = json.load(f)["data"]
            T = np.array(pose_data).reshape(4, 4)
            R = T[:3, :3]
            rotations.append(R)
    return rotations

In [102]:
#affine
def run_detailed_stitcher(image_paths, output_path="result.jpg"):
    cmd = [
        "python", "stitching_detailed.py",
        *image_paths,
        "--features", "sift",
        "--matcher", "affine",
        "--estimator", "affine",
        "--match_conf", "0.3",
        "--conf_thresh", "0.3",
        "--ba", "affine",
        "--ba_refine_mask", "xxxxx",
        "--wave_correct", "no",
        "--warp", "affine",
        "--blend", "feather",
        "--blend_strength", "5",
        "--work_megapix", "0.6",
        "--seam_megapix", "0.07",
        "--compose_megapix", "0.3",
        "--output", output_path,
    ]

    print("🛠️ Running affine-mode stitcher:\n", " ".join(cmd))

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Saved stitched result to {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Stitcher failed: {e}")


In [108]:
#cylindrical
def run_detailed_stitcher(image_paths, output_path="result.jpg"):
    cmd = [
        "python", "stitching_detailed.py",
        *image_paths,
        "--features", "sift",
        "--matcher", "homography",
        "--estimator", "homography",
        "--blend", "feather",
        "--blend_strength", "5",
        "--warp", "cylindrical",
        "--wave_correct", "no",
        "--conf_thresh", "0.25",
        "--match_conf", "0.25",
        "--work_megapix", "0.3",
        "--seam_megapix", "0.07",
        "--compose_megapix", "0.2",
        "--output", output_path,
    ]

    print("🛠️ Running command:\n", " ".join(cmd))

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Saved stitched result to {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Stitcher failed: {e}")


In [69]:
#corrected matrices + cv2 sititcher
def create_pano(pano_path):
    images, image_paths = get_image_folder(pano_path)
    intrinsics = get_intrinsics(pano_path)
    rotations = get_rotations(pano_path)

    # Try OpenCV Stitcher first
    stitcher = cv2.Stitcher_create()
    status, pano = stitcher.stitch(images)

    if status == cv2.Stitcher_OK:
        print(f"✅ Stitch succeeded for: {pano_path}")
        return pano
    else:
        print(f"❌ Stitch failed, falling back to homography for: {pano_path}")
        # Manual stitching with homographies
        base_img = images[0]
        result = base_img.copy()

        for i in range(1, len(images)):
            image_shape = images[i].shape
            R_ref = pose_rotation(rotations[i - 1])
            R_i = pose_rotation(rotations[i])
            K_i = rotate_instrinsics_90cw(intrinsics[i], image_shape)

            # Relative rotation
            R_rel = R_i @ R_ref.T
            H = K_i @ R_rel @ np.linalg.inv(K_i)

            warped_img = cv2.warpPerspective(images[i], H, (result.shape[1], result.shape[0]))
            result = cv2.addWeighted(result, 0.5, warped_img, 0.5, 0)

        return result



In [81]:
pano_path = "../PALMS+/SVC/Session_1744240794"
images, image_paths = get_image_folder(pano_path)
rotated_images = get_rotated_image_paths_from_palms_path(pano_path)
intrinsics = get_intrinsics(pano_path)
rotations = get_rotations(pano_path)

base_img = images[0]
result = base_img.copy()

for i in range(1, len(images)):
    image_shape = images[i].shape
    R_ref = pose_rotation(rotations[i - 1])
    R_i = pose_rotation(rotations[i])
    K_i = rotate_instrinsics_90cw(intrinsics[i], image_shape)

    # Relative rotation
    R_rel = R_i @ R_ref.T
    H = K_i @ R_rel @ np.linalg.inv(K_i)

    warped_img = cv2.warpPerspective(images[i], H, (result.shape[1], result.shape[0]))
    result = cv2.addWeighted(result, 0.5, warped_img, 0.5, 0)

cv2.imwrite("result.jpg", result)

True

In [98]:
pano_path = "../PALMS+/BE/Session_1744229291"
image_paths = get_image_paths(pano_path)

# Optionally rotate and save if your images are portrait but stored as landscape
rotated_paths = []
for i, path in enumerate(image_paths):
    img = cv2.imread(path)
    img_rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    rotated_path = f"temp_rotated_{i}.png"
    cv2.imwrite(rotated_path, img_rotated)
    rotated_paths.append(rotated_path)

# Run the advanced stitcher on the rotated paths

run_detailed_stitcher(rotated_paths, output_path="stitched_result.jpg")

python(33039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


SURF not available
❌ Stitcher failed: Command '['python', 'stitching_detailed.py', 'temp_rotated_0.png', 'temp_rotated_1.png', 'temp_rotated_2.png', 'temp_rotated_3.png', 'temp_rotated_4.png', 'temp_rotated_5.png', 'temp_rotated_6.png', 'temp_rotated_7.png', 'temp_rotated_8.png', 'temp_rotated_9.png', 'temp_rotated_10.png', 'temp_rotated_11.png', 'temp_rotated_12.png', 'temp_rotated_13.png', 'temp_rotated_14.png', 'temp_rotated_15.png', 'temp_rotated_16.png', 'temp_rotated_17.png', '--features', 'sift', '--matcher', 'homography', '--estimator', 'homography', '--blend', 'multiband', '--warp', 'cylindrical', '--wave_correct', 'no', '--conf_thresh', '0.2', '--match_conf', '0.2', '--seam', 'gc_colorgrad--output', 'stitched_result.jpg', '--work_megapix', '0.3', '--seam_megapix', '0.07', '--compose_megapix', '0.3']' returned non-zero exit status 2.


usage: stitching_detailed.py [-h] [--try_cuda TRY_CUDA]
                             [--work_megapix WORK_MEGAPIX]
                             [--features {orb,sift,brisk,akaze}]
                             [--matcher {homography,affine}]
                             [--estimator {homography,affine}]
                             [--match_conf MATCH_CONF]
                             [--conf_thresh CONF_THRESH]
                             [--ba {ray,reproj,affine,no}]
                             [--ba_refine_mask BA_REFINE_MASK]
                             [--wave_correct {horiz,no,vert}]
                             [--save_graph SAVE_GRAPH]
                             [--warp {spherical,plane,affine,cylindrical,fisheye,stereographic,compressedPlaneA2B1,compressedPlaneA1.5B1,compressedPlanePortraitA2B1,compressedPlanePortraitA1.5B1,paniniA2B1,paniniA1.5B1,paniniPortraitA2B1,paniniPortraitA1.5B1,mercator,transverseMercator}]
                             [--seam_megapix SEAM_MEGAPI

In [83]:
#corrected matrices + cv2 sititcher
def create_pano(pano_path):
    images, image_paths = get_image_folder(pano_path)
    rotated_paths = get_rotated_image_paths_from_palms_path(pano_path)
    intrinsics = get_intrinsics(pano_path)
    rotations = get_rotations(pano_path)

    #create pano name
    pano_name = os.path.basename(pano_path) + ".jpg"
    run_detailed_stitcher(rotated_paths, output_path=pano_name)


    if os.path.exists(pano_name):
        print(f"✅ Stitch succeeded for: {pano_path}")
        return cv2.imread(pano_name)
    else:
        print(f"❌ Stitch failed, falling back to homography for: {pano_path}")
        # Manual stitching with homographies
        base_img = images[0]
        result = base_img.copy()

        for i in range(1, len(images)):
            image_shape = images[i].shape
            R_ref = pose_rotation(rotations[i - 1])
            R_i = pose_rotation(rotations[i])
            K_i = rotate_instrinsics_90cw(intrinsics[i], image_shape)

            # Relative rotation
            R_rel = R_i @ R_ref.T
            H = K_i @ R_rel @ np.linalg.inv(K_i)

            warped_img = cv2.warpPerspective(images[i], H, (result.shape[1], result.shape[0]))
            result = cv2.addWeighted(result, 0.5, warped_img, 0.5, 0)
        # Save the final result
        cv2.imwrite(pano_name, result)

        return result

In [64]:
def create_pano(pano_path):
    import os

    # Paths
    building = os.path.basename(os.path.dirname(pano_path))
    session = os.path.basename(pano_path)
    pano_name = os.path.join(building, f"{session}_stitched.jpg")  # Save to Panorama/BE/session_stitched.jpg

    # Rotated images from Panorama/BE/Session_*/image_*.png
    rotated_paths = get_rotated_image_paths_from_palms_path(pano_path)
    images = [cv2.imread(p) for p in rotated_paths]

    # Load original intrinsics and poses from PALMS+
    intrinsics = get_intrinsics(pano_path)
    rotations = get_rotations(pano_path)

    # Try stitcher
    run_detailed_stitcher(rotated_paths, output_path=pano_name)

    if os.path.exists(pano_name):
        print(f"✅ Stitch succeeded: {pano_name}")
        return cv2.imread(pano_name)
    else:
        print(f"❌ Stitch failed, falling back to homography: {pano_name}")
        base_img = images[0]
        result = base_img.copy()

        for i in range(1, len(images)):
            image_shape = images[i].shape
            R_ref = pose_rotation(rotations[i - 1])
            R_i = pose_rotation(rotations[i])
            K_i = rotate_instrinsics_90cw(intrinsics[i], image_shape)

            R_rel = R_i @ R_ref.T
            H = K_i @ R_rel @ np.linalg.inv(K_i)

            warped_img = cv2.warpPerspective(images[i], H, (result.shape[1], result.shape[0]))
            result = cv2.addWeighted(result, 0.5, warped_img, 0.5, 0)

        cv2.imwrite(pano_name, result)
        print(f"✅ Fallback panorama saved: {pano_name}")
        return result


In [109]:
if __name__ == "__main__":
    all_sessions = get_pano_file_paths()
    for path in all_sessions:
        print(f"\n📂 Processing session: {path}")
        try:
            create_pano(path)
        except Exception as e:
            print(f"❌ Failed on {path}: {e}")


📂 Processing session: ../Palms+/BE/Session_1744229291
🛠️ Running command:
 python stitching_detailed.py BE/Session_1744229291/image_0031831.png BE/Session_1744229291/image_0031834.png BE/Session_1744229291/image_0031836.png BE/Session_1744229291/image_0031838.png BE/Session_1744229291/image_0031840.png BE/Session_1744229291/image_0031842.png BE/Session_1744229291/image_0031843.png BE/Session_1744229291/image_0031845.png BE/Session_1744229291/image_0031846.png BE/Session_1744229291/image_0031848.png BE/Session_1744229291/image_0031849.png BE/Session_1744229291/image_0031851.png BE/Session_1744229291/image_0031852.png BE/Session_1744229291/image_0031853.png BE/Session_1744229291/image_0031855.png BE/Session_1744229291/image_0031856.png BE/Session_1744229291/image_0031857.png BE/Session_1744229291/image_0031858.png --features sift --matcher homography --estimator homography --blend feather --blend_strength 5 --warp cylindrical --wave_correct no --conf_thresh 0.25 --match_conf 0.25 --work

python(35053) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


SURF not available
['BE/Session_1744229291/image_0031831.png', 'BE/Session_1744229291/image_0031834.png', 'BE/Session_1744229291/image_0031836.png', 'BE/Session_1744229291/image_0031838.png', 'BE/Session_1744229291/image_0031840.png', 'BE/Session_1744229291/image_0031842.png', 'BE/Session_1744229291/image_0031843.png', 'BE/Session_1744229291/image_0031845.png', 'BE/Session_1744229291/image_0031846.png', 'BE/Session_1744229291/image_0031848.png', 'BE/Session_1744229291/image_0031849.png', 'BE/Session_1744229291/image_0031851.png', 'BE/Session_1744229291/image_0031852.png', 'BE/Session_1744229291/image_0031853.png', 'BE/Session_1744229291/image_0031855.png', 'BE/Session_1744229291/image_0031856.png', 'BE/Session_1744229291/image_0031857.png', 'BE/Session_1744229291/image_0031858.png']
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  5 had an illegal value
** On entry to

python(35411) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


SURF not available
['BE/Session_1744230038/image_0032575.png', 'BE/Session_1744230038/image_0032578.png', 'BE/Session_1744230038/image_0032580.png', 'BE/Session_1744230038/image_0032581.png', 'BE/Session_1744230038/image_0032582.png', 'BE/Session_1744230038/image_0032584.png', 'BE/Session_1744230038/image_0032586.png', 'BE/Session_1744230038/image_0032588.png', 'BE/Session_1744230038/image_0032590.png', 'BE/Session_1744230038/image_0032592.png', 'BE/Session_1744230038/image_0032594.png', 'BE/Session_1744230038/image_0032596.png', 'BE/Session_1744230038/image_0032598.png', 'BE/Session_1744230038/image_0032599.png', 'BE/Session_1744230038/image_0032601.png', 'BE/Session_1744230038/image_0032602.png', 'BE/Session_1744230038/image_0032604.png', 'BE/Session_1744230038/image_0032605.png', 'BE/Session_1744230038/image_0032606.png']


...

python(36351) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


SURF not available
['PS/Session_1744231073/image_0033593.png', 'PS/Session_1744231073/image_0033598.png', 'PS/Session_1744231073/image_0033601.png', 'PS/Session_1744231073/image_0033602.png', 'PS/Session_1744231073/image_0033604.png', 'PS/Session_1744231073/image_0033607.png', 'PS/Session_1744231073/image_0033608.png', 'PS/Session_1744231073/image_0033610.png', 'PS/Session_1744231073/image_0033611.png', 'PS/Session_1744231073/image_0033613.png', 'PS/Session_1744231073/image_0033615.png', 'PS/Session_1744231073/image_0033616.png', 'PS/Session_1744231073/image_0033618.png', 'PS/Session_1744231073/image_0033619.png', 'PS/Session_1744231073/image_0033620.png', 'PS/Session_1744231073/image_0033622.png', 'PS/Session_1744231073/image_0033623.png', 'PS/Session_1744231073/image_0033625.png']
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  5 had an illegal value
** On entry to

python(36660) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


SURF not available
['PS/Session_1744231837/image_0034353.png', 'PS/Session_1744231837/image_0034358.png', 'PS/Session_1744231837/image_0034359.png', 'PS/Session_1744231837/image_0034361.png', 'PS/Session_1744231837/image_0034362.png', 'PS/Session_1744231837/image_0034363.png', 'PS/Session_1744231837/image_0034365.png', 'PS/Session_1744231837/image_0034367.png', 'PS/Session_1744231837/image_0034369.png', 'PS/Session_1744231837/image_0034373.png', 'PS/Session_1744231837/image_0034375.png', 'PS/Session_1744231837/image_0034377.png', 'PS/Session_1744231837/image_0034379.png', 'PS/Session_1744231837/image_0034381.png', 'PS/Session_1744231837/image_0034384.png', 'PS/Session_1744231837/image_0034386.png', 'PS/Session_1744231837/image_0034388.png', 'PS/Session_1744231837/image_0034389.png']
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  4 had an illegal value
** On entry to DLASCL, parameter number  5 had an illegal value
** On entry to

KeyboardInterrupt: 

In [118]:
def has_enough_matches(img1, img2, min_matches=10):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()

    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    if des1 is None or des2 is None:
        return False

    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(des1, des2, k=2)

    # Ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return len(good_matches) >= min_matches

def stitch_with_homography(base_img, next_img, R_ref, R_i, K_i):
    R_rel = R_i @ R_ref.T
    H = K_i @ R_rel @ np.linalg.inv(K_i)
    warped = cv2.warpPerspective(next_img, H, (base_img.shape[1], base_img.shape[0]))
    return cv2.addWeighted(base_img, 0.5, warped, 0.5, 0)

def create_pano_pairwise(pano_path):
    building = os.path.basename(os.path.dirname(pano_path))
    session = os.path.basename(pano_path)
    panorama_dir = os.path.join("Panorama", building)
    os.makedirs(panorama_dir, exist_ok=True)
    output_path = os.path.join(panorama_dir, f"{session}_stitched.jpg")

    rotated_paths = get_rotated_image_paths_from_palms_path(pano_path)
    images = [cv2.imread(p) for p in rotated_paths]
    intrinsics = get_intrinsics(pano_path)
    rotations = get_rotations(pano_path)
    #exlude first and last images
    images = images[1:-1]
    intrinsics = intrinsics[1:-1]
    rotations = rotations[1:-1]
    rotated_paths = rotated_paths[1:-1]

    result = images[0]
    R_ref = pose_rotation(rotations[0])

    for i in range(1, len(images)):
        print(f"🔗 Stitching image {i-1} to {i}...")
        if has_enough_matches(result, images[i]):
            print("✅ Enough features: using keypoints")
            stitcher = cv2.Stitcher_create()
            status, pano = stitcher.stitch([result, images[i]])
            if status == cv2.Stitcher_OK:
                result = pano
                continue
            else:
                print("⚠️ Stitcher failed, falling back to homography")

        print("🔁 Using pose + intrinsics fallback")
        R_i = pose_rotation(rotations[i])
        image_shape = images[i].shape
        K_i = rotate_instrinsics_90cw(intrinsics[i], image_shape)
        result = stitch_with_homography(result, images[i], R_ref, R_i, K_i)
        R_ref = R_i  # update reference rotation

    result_uint8 = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX).astype("uint8")
    output_filename = f"{session}_stitched.jpg"
    panorama_path = os.path.join("Panorama", building, output_filename)
    cv2.imwrite(panorama_path, result_uint8)
    print(f"✅ Final stitched panorama saved to {panorama_path}")
    return result_uint8


In [119]:
if __name__ == "__main__":
    all_sessions = get_pano_file_paths()
    for path in all_sessions:
        print(f"\n📂 Processing session: {path}")
        try:
            create_pano_pairwise(path)
        except Exception as e:
            print(f"❌ Failed on {path}: {e}")


📂 Processing session: ../Palms+/BE/Session_1744229291
🔗 Stitching image 0 to 1...
✅ Enough features: using keypoints
🔗 Stitching image 1 to 2...
✅ Enough features: using keypoints
🔗 Stitching image 2 to 3...
✅ Enough features: using keypoints
🔗 Stitching image 3 to 4...
✅ Enough features: using keypoints
🔗 Stitching image 4 to 5...
✅ Enough features: using keypoints
🔗 Stitching image 5 to 6...
✅ Enough features: using keypoints
🔗 Stitching image 6 to 7...
✅ Enough features: using keypoints
🔗 Stitching image 7 to 8...
✅ Enough features: using keypoints
🔗 Stitching image 8 to 9...
✅ Enough features: using keypoints
⚠️ Stitcher failed, falling back to homography
🔁 Using pose + intrinsics fallback
🔗 Stitching image 9 to 10...
✅ Enough features: using keypoints
⚠️ Stitcher failed, falling back to homography
🔁 Using pose + intrinsics fallback
🔗 Stitching image 10 to 11...
✅ Enough features: using keypoints
⚠️ Stitcher failed, falling back to homography
🔁 Using pose + intrinsics fallback
🔗 S